# Credit Card Fraud Detection::

Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

# Description about dataset::

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import models,layers
from keras import regularizers


In [3]:
df = pd.read_csv('./creditcard.csv')
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
non_fraud=df[df["Class"]==0]
fraud = df[df["Class"]==1]
non_fraud=non_fraud.sample(3*fraud.shape[0])
data = fraud.append(non_fraud, ignore_index=True)
# data["Class"].value_counts()
x_data= data.drop(columns="Class", axis=0)
label = data["Class"]

In [5]:
train_data, test_data, train_labels, test_labels=train_test_split(x_data, label, test_size=0.3, random_state=1, stratify = label)

In [6]:
train_mean = train_data.mean(axis=0) # taking the mean of 
train_data -= train_mean
train_std = train_data.std(axis=0)
train_data /= train_std
test_data -= train_mean
test_data /= train_std

In [7]:
test_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
1370,1.479278,0.790737,-0.519405,0.062660,-0.729994,0.121302,-0.735743,0.327838,-0.216164,-0.132621,...,0.106604,-0.071658,0.220988,0.181418,0.100259,-0.046038,-0.701557,-0.125144,-0.134283,-0.174668
265,-0.169031,0.084833,0.855763,-0.959624,1.379723,-0.452715,-1.246308,-1.005621,0.336598,-1.954056,...,1.115828,0.401005,-0.499256,-0.257300,-0.608236,1.532433,2.433244,1.586640,1.717480,-0.432897
289,-0.016866,0.745137,0.236788,-0.528218,0.453924,0.892547,-0.799513,0.470865,-0.110370,0.995623,...,-0.262831,-0.343130,-0.967257,-0.005484,0.037020,0.612384,-1.225780,-0.037371,0.183483,-0.428286
644,-1.170784,0.068908,-0.010341,0.583101,0.087551,0.188865,0.231424,0.442799,0.036334,0.306749,...,-0.065826,-0.145868,0.265251,0.160837,0.184927,-0.134201,-0.545418,-0.001546,-0.308285,-0.168766
1818,0.513072,0.411285,-0.227943,0.451921,-0.014194,-0.070914,0.745837,0.308934,-0.022095,0.551685,...,0.015356,0.019876,1.110197,0.015618,-0.738152,-1.310357,2.048378,-0.262108,-0.255144,0.099470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,-1.445346,0.578215,-0.427996,0.520287,-0.657729,0.020820,0.201683,0.145615,-0.036463,2.023808,...,-0.197021,-0.193600,0.072834,-0.025312,-0.771765,0.793909,-1.420396,0.028331,0.041162,-0.378254
1822,0.636217,0.752836,-0.219208,-0.035949,-0.020752,0.499821,-0.354785,0.502363,-0.168888,0.079198,...,-0.199389,-0.025878,0.647426,-0.047213,-0.554945,0.822513,-1.042556,-0.108689,-0.178451,-0.283032
210,-0.620890,-0.921096,-0.410212,-0.425563,1.526487,-2.116270,2.646582,0.618735,0.004787,-1.251858,...,-1.612451,0.019257,1.426299,-0.902826,-0.043714,-0.566902,0.906120,1.762561,-2.652046,6.032798
322,0.054394,0.491116,-0.437148,-0.144859,1.339371,0.478558,-0.187718,0.636965,-0.207650,0.282712,...,0.928701,-0.167446,-1.259831,-0.261470,-0.632135,-0.370313,-0.589770,-0.242289,0.236944,2.202334


In [8]:


model = models.Sequential()
model.add(layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [9]:
model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [10]:
model.fit(train_data, train_labels, epochs=50,validation_split = 0.3)

Epoch 1/50
31/31 [==============================] - 2s 32ms/step - loss: 0.8316 - accuracy: 0.5881 - val_loss: 0.7031 - val_accuracy: 0.7391
Epoch 2/50
31/31 [==============================] - 0s 3ms/step - loss: 0.7815 - accuracy: 0.6513 - val_loss: 0.6817 - val_accuracy: 0.7391
Epoch 3/50
31/31 [==============================] - 0s 3ms/step - loss: 0.7600 - accuracy: 0.7055 - val_loss: 0.6706 - val_accuracy: 0.7391
Epoch 4/50
31/31 [==============================] - 0s 2ms/step - loss: 0.7275 - accuracy: 0.6860 - val_loss: 0.6529 - val_accuracy: 0.7391
Epoch 5/50
31/31 [==============================] - 0s 2ms/step - loss: 0.7118 - accuracy: 0.7033 - val_loss: 0.6320 - val_accuracy: 0.7391
Epoch 6/50
31/31 [==============================] - 0s 3ms/step - loss: 0.6640 - accuracy: 0.7243 - val_loss: 0.6085 - val_accuracy: 0.7391
Epoch 7/50
31/31 [==============================] - 0s 2ms/step - loss: 0.6493 - accuracy: 0.7394 - val_loss: 0.5928 - val_accuracy: 0.7391
Epoch 8/50
31/31 [=

In [11]:
loss, accuracy = model.evaluate(test_data, test_labels)

19/19 [==============================] - 0s 889us/step - loss: 0.2200 - accuracy: 0.9543


In [12]:
loss

0.22000302374362946

In [13]:
accuracy

0.9543147087097168

In [28]:
predictions = model.predict(test_data)

In [29]:
y_pred = (predictions > 0.5)

In [30]:
con = tf.math.confusion_matrix(
    test_labels, y_pred, num_classes=2, weights=None, dtype=tf.dtypes.int32,
    name=None
)

In [31]:
con

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[440,   3],
       [ 24, 124]])>

In [32]:
result = accuracy * 100
result

95.43147087097168